**Load the required packages and data**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e20/sample_submission.csv
/kaggle/input/playground-series-s3e20/train.csv
/kaggle/input/playground-series-s3e20/test.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
# from sklearn.neighbors impor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
from sklearn.decomposition import PCA

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Load the datasets 
df = pd.read_csv("/kaggle/input/playground-series-s3e20/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s3e20/test.csv")
sample_submission = pd.read_csv("/kaggle/input/playground-series-s3e20/sample_submission.csv")

# Explanatory Data Analysis
## Data Assessment

In [4]:
df.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,...,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,...,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,...,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296,4.231381
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,NaN,NaN,NaN,NaN,NaN,...,5228.507736,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155,4.305286
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,...,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612,4.347317


In [5]:
test_df.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
0,ID_-0.510_29.290_2022_00,-0.51,29.29,2022,0,NaN,NaN,NaN,NaN,NaN,...,36022.027344,8472.313477,41047.937500,7472.313477,7.935617,0.240773,-100.113792,33.697044,-133.047546,33.779583
1,ID_-0.510_29.290_2022_01,-0.51,29.29,2022,1,0.000456,0.691164,0.000316,0.000000,76.239196,...,48539.737242,6476.147323,54915.708579,5476.147161,11.448437,0.293119,-30.510319,42.402593,-138.632822,31.012380
2,ID_-0.510_29.290_2022_02,-0.51,29.29,2022,2,0.000161,0.605107,0.000106,0.079870,-42.055341,...,34133.080469,8984.795703,39006.093750,7984.795703,10.753179,0.267130,39.087361,45.936480,-144.784988,26.743361
3,ID_-0.510_29.290_2022_03,-0.51,29.29,2022,3,0.000350,0.696917,0.000243,0.201028,72.169566,...,50854.991076,6014.724059,57646.368368,5014.724115,11.764556,0.304679,-24.465127,42.140419,-135.027891,29.604774
4,ID_-0.510_29.290_2022_04,-0.51,29.29,2022,4,-0.000317,0.580527,-0.000184,0.204352,76.190865,...,46594.685145,6849.280477,52896.541873,5849.280394,13.065317,0.284221,-12.907850,30.122641,-135.500119,26.276807


In [6]:
df.columns

Index(['ID_LAT_LON_YEAR_WEEK', 'latitude', 'longitude', 'year', 'week_no',
       'SulphurDioxide_SO2_column_number_density',
       'SulphurDioxide_SO2_column_number_density_amf',
       'SulphurDioxide_SO2_slant_column_number_density',
       'SulphurDioxide_cloud_fraction', 'SulphurDioxide_sensor_azimuth_angle',
       'SulphurDioxide_sensor_zenith_angle',
       'SulphurDioxide_solar_azimuth_angle',
       'SulphurDioxide_solar_zenith_angle',
       'SulphurDioxide_SO2_column_number_density_15km',
       'CarbonMonoxide_CO_column_number_density',
       'CarbonMonoxide_H2O_column_number_density',
       'CarbonMonoxide_cloud_height', 'CarbonMonoxide_sensor_altitude',
       'CarbonMonoxide_sensor_azimuth_angle',
       'CarbonMonoxide_sensor_zenith_angle',
       'CarbonMonoxide_solar_azimuth_angle',
       'CarbonMonoxide_solar_zenith_angle',
       'NitrogenDioxide_NO2_column_number_density',
       'NitrogenDioxide_tropospheric_NO2_column_number_density',
       'NitrogenDioxide

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 76 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   ID_LAT_LON_YEAR_WEEK                                      79023 non-null  object 
 1   latitude                                                  79023 non-null  float64
 2   longitude                                                 79023 non-null  float64
 3   year                                                      79023 non-null  int64  
 4   week_no                                                   79023 non-null  int64  
 5   SulphurDioxide_SO2_column_number_density                  64414 non-null  float64
 6   SulphurDioxide_SO2_column_number_density_amf              64414 non-null  float64
 7   SulphurDioxide_SO2_slant_column_number_density            64414 non-null  float64
 8   SulphurDioxide_c

## Data Wrangling 

In [8]:
# Remove the id column 
df.drop("ID_LAT_LON_YEAR_WEEK", axis=1, inplace=True)
df.columns

Index(['latitude', 'longitude', 'year', 'week_no',
       'SulphurDioxide_SO2_column_number_density',
       'SulphurDioxide_SO2_column_number_density_amf',
       'SulphurDioxide_SO2_slant_column_number_density',
       'SulphurDioxide_cloud_fraction', 'SulphurDioxide_sensor_azimuth_angle',
       'SulphurDioxide_sensor_zenith_angle',
       'SulphurDioxide_solar_azimuth_angle',
       'SulphurDioxide_solar_zenith_angle',
       'SulphurDioxide_SO2_column_number_density_15km',
       'CarbonMonoxide_CO_column_number_density',
       'CarbonMonoxide_H2O_column_number_density',
       'CarbonMonoxide_cloud_height', 'CarbonMonoxide_sensor_altitude',
       'CarbonMonoxide_sensor_azimuth_angle',
       'CarbonMonoxide_sensor_zenith_angle',
       'CarbonMonoxide_solar_azimuth_angle',
       'CarbonMonoxide_solar_zenith_angle',
       'NitrogenDioxide_NO2_column_number_density',
       'NitrogenDioxide_tropospheric_NO2_column_number_density',
       'NitrogenDioxide_stratospheric_NO2_colum

In [9]:
# drop columns with more than 30% null values 
limitPer = len(df) * .15
df.dropna(thresh=limitPer, axis=1, inplace=True)

In [10]:
# drop all null values then
df.dropna(axis=0, how='any', inplace=True)
# confirm the operation
df.isna().sum()

latitude                                    0
longitude                                   0
year                                        0
week_no                                     0
SulphurDioxide_SO2_column_number_density    0
                                           ..
Cloud_sensor_azimuth_angle                  0
Cloud_sensor_zenith_angle                   0
Cloud_solar_azimuth_angle                   0
Cloud_solar_zenith_angle                    0
emission                                    0
Length: 68, dtype: int64

In [11]:
df.shape

(57209, 68)

## Data Analysis and Visualization

# Machine Learning
## Data Preprocessing 

**Split into dependent and independent datasets**

In [12]:
X = df.drop("emission", axis=1)
y = df["emission"]

In [13]:
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer = imputer.fit(X)
  
# X = imputer.transform(X)
# X

**Split the data into train and validation datasets**

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

## Models 
### Linear Regression


In [15]:
# Fit the model to train dataset
lr = LinearRegression()

lr.fit(X_train, y_train)

LinearRegression()

In [16]:
# Do the predictions
y_pred = lr.predict(X_val)
y_pred

array([ 58.34665626,  66.87070613,  46.76158372, ..., 129.84086159,
        47.15681692,  74.91671485])

**Evaluate linear regression**

In [17]:
# Accuracy
r2_score(y_val, y_pred)

0.032788210218455904

In [18]:
sqrt(mean_squared_error(y_val, y_pred))

134.77064479900525

### Support Vector Machine regressor

In [19]:
# # Fit the model to train dataset
# svr = SVR()

# svr.fit(X_train, y_train)

**Evaluate SVM regressor**

In [20]:
# # Do the predictions
# y_pred = svr.predict(X_val)

# # Accuracy
# r2_score(y_val, y_pred)

In [21]:
# sqrt(mean_squared_error(y_val, y_pred))

### Random Forest Regressor

In [22]:
# Fit the model to train dataset
rf = RandomForestRegressor()

rf.fit(X_train, y_train)

RandomForestRegressor()

**Evaluate the Random Forest regressor**

In [23]:
# Do the predictions
y_pred = rf.predict(X_val)

# Accuracy
r2_score(y_val, y_pred)# Do the predictions

0.9681413883550515

In [24]:
sqrt(mean_squared_error(y_val, y_pred))

24.459520464733156

### Feature importance
So far random forest regressor has the best accuracy. Lets find the most important features in the dataset. 

In [25]:
from sklearn.inspection import permutation_importance

# rf = RandomForestRegressor(n_estimators=100, random_state=42)
# rf.fit(X_train, y_train)

result = permutation_importance(rf, X_val, y_val, n_repeats=10, random_state=42)



In [26]:
perm_importances = result.importances_mean
perm_std = result.importances_std
sorted_idx = perm_importances.argsort()
feature_names = X_val.columns

feature_importance_df = pd.DataFrame({
    'Importance': perm_importances, 
    'Std': perm_std
}, index=feature_names[sorted_idx]).sort_values('Importance',ascending=False)

feature_importance_df

,Importance,Std
NitrogenDioxide_solar_zenith_angle,1.619906,0.054685
Ozone_cloud_fraction,0.588582,0.054021
UvAerosolIndex_sensor_azimuth_angle,0.160881,0.047008
NitrogenDioxide_NO2_column_number_density,0.003647,0.000311
Formaldehyde_tropospheric_HCHO_column_number_density_amf,0.002622,0.000152
...,...,...
Formaldehyde_solar_azimuth_angle,-0.000044,0.000014
Ozone_solar_zenith_angle,-0.000046,0.000028
CarbonMonoxide_sensor_zenith_angle,-0.000047,0.000033
SulphurDioxide_SO2_column_number_density_amf,-0.000053,0.000028


#### $1^{st}$ Trial
Features that have importance more than zero will be selected


In [27]:
selected_features = feature_importance_df[feature_importance_df["Importance"]>0].index.values

In [28]:
X_train1  = X_train[selected_features]
X_val1  = X_val[selected_features]

In [29]:
# Fit the model to train dataset
rf = RandomForestRegressor()
rf.fit(X_train1, y_train)

# Do the predictions
y_pred = rf.predict(X_val1)

# Accuracy
r2_score(y_val, y_pred)# Do the predictions

0.9695923594846481

In [30]:
sqrt(mean_squared_error(y_val, y_pred))

23.896036696404764

#### $2^{nd}$ Trial
Features that have importance more than 0.1 will be selected

In [31]:
selected_features = feature_importance_df[feature_importance_df["Importance"]>0.1].index.values

In [32]:
X_train2  = X_train[selected_features]
X_val2  = X_val[selected_features]

In [33]:
# Fit the model to train dataset
rf = RandomForestRegressor()
rf.fit(X_train2, y_train)

# Do the predictions
y_pred = rf.predict(X_val2)

# Accuracy
r2_score(y_val, y_pred)# Do the predictions

-0.04609822545613551

In [34]:
sqrt(mean_squared_error(y_val, y_pred))

140.15892183894286

### Correlation

Calculate correlation and select features that are more correlated with "emission" variable. 

In [35]:
# %matplotlib inline
# sns.set()

# # Plotting
# plt.figure(figsize=(7, 7))
# sns.heatmap(df.corr(), annot=True, cmap="Greens")
# plt.show()

In [36]:
corr_df =  df.corr()

selected_features = corr_df[(corr_df["emission"]>0.01)|(corr_df["emission"]<-0.01)].index.values
selected_features[:-1]

array(['latitude', 'longitude', 'year',
       'SulphurDioxide_SO2_column_number_density',
       'SulphurDioxide_SO2_slant_column_number_density',
       'SulphurDioxide_sensor_azimuth_angle',
       'SulphurDioxide_solar_azimuth_angle',
       'SulphurDioxide_SO2_column_number_density_15km',
       'CarbonMonoxide_CO_column_number_density',
       'CarbonMonoxide_H2O_column_number_density',
       'CarbonMonoxide_cloud_height',
       'CarbonMonoxide_solar_azimuth_angle',
       'CarbonMonoxide_solar_zenith_angle',
       'NitrogenDioxide_stratospheric_NO2_column_number_density',
       'NitrogenDioxide_NO2_slant_column_number_density',
       'NitrogenDioxide_absorbing_aerosol_index',
       'NitrogenDioxide_cloud_fraction',
       'NitrogenDioxide_sensor_altitude',
       'NitrogenDioxide_sensor_azimuth_angle',
       'NitrogenDioxide_solar_azimuth_angle',
       'Formaldehyde_tropospheric_HCHO_column_number_density',
       'Formaldehyde_tropospheric_HCHO_column_number_density_amf

In [37]:
X_train3  = X_train[selected_features[:-1]]
X_val3  = X_val[selected_features[:-1]]

In [38]:
# Fit the model to train dataset
rf = RandomForestRegressor()
rf.fit(X_train3, y_train)

# Do the predictions
y_pred = rf.predict(X_val3)

# Accuracy
r2_score(y_val, y_pred)# Do the predictions

0.923724527219888

In [39]:
sqrt(mean_squared_error(y_val, y_pred))

37.84660865515413

### Dimensionality Reduction - PCA

In [40]:
pca10 = PCA(n_components=10)

# transform the variables
X_train4 = pca10.fit_transform(X_train)
X_val4 = pca10.transform(X_val)

# Fit the model to train dataset
rf = RandomForestRegressor()
rf.fit(X_train4, y_train)

# Do the predictions
y_pred = rf.predict(X_val4)

# Accuracy
r2_score(y_val, y_pred)

-0.007198184279317843

In [41]:
sqrt(mean_squared_error(y_val, y_pred))

137.52827079991866

In [42]:
pca10 = PCA(n_components=10)

# transform the variables
X_train4 = pca10.fit_transform(X_train)
X_val4 = pca10.transform(X_val)

# Fit the model to train dataset
lr.fit(X_train4, y_train)

# Do the predictions
y_pred = lr.predict(X_val4)

# Accuracy
r2_score(y_val, y_pred)

0.007728841685365362

In [43]:
sqrt(mean_squared_error(y_val, y_pred))

136.50535836837764

### MultiLayer Perceptron

In [44]:
# stop here

In [45]:
# Model Creation

In [46]:
# Fit to train data

**Evaluate the model**

# Submission

In [47]:
# test_df = test_df[X.columns]
# test_df.head()

In [48]:
# test_df.isna().sum()

In [49]:
# for col in test_df.columns:
#     var_mean = test_df[col].mean()
#     test_df[col].fillna(var_mean, inplace=True)
    
# test_df.isna().sum().sum()

In [50]:
# predictions = rf.predict(test_df)

In [51]:
# test_data = pd.read_csv("/kaggle/input/playground-series-s3e20/test.csv")

In [52]:
# id_var = test_data['ID_LAT_LON_YEAR_WEEK']
# assert len(id_var)==len(predictions)

In [53]:
# submission_df = pd.DataFrame({
#     'ID_LAT_LON_YEAR_WEEK':id_var,
#     'emission':predictions
# })

# submission_df.head()

In [54]:
# submission_df.to_csv('submission.csv', index = False)

In [55]:
# submission_df.to_csv('/kaggle/working/submission.csv', index = False)